In [87]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-nano")

llm.invoke("hi")

AIMessage(content='Hi! Nice to meet you. What would you like help with today? I can answer questions, explain concepts, help with writing or coding, plan something, or just chat. Tell me what you’re interested in.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 437, 'prompt_tokens': 7, 'total_tokens': 444, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D0KgkugbNRHt0bOxFtnY1s4CJi5ci', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdef1-0ce9-7963-a155-f8a2d2b1ed26-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 437, 'total_tokens': 444, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [88]:
# Initialize tool calls
from langchain_core.tools import tool
from typing import Literal
from datetime import datetime
from pydantic import BaseModel

@tool
def write_email(to:str, subject:str, thread:str)->str:
    """Write and send email"""
    return f"Email has been send to user : {to}"

@tool
def schedule_meeting(attendies:list[str], subject:str, duration_min:int, preferred_day:datetime, start_time:str)->str:
    """Schedule a calender meeting"""
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
    """Email has been send"""
    done:bool
    

In [109]:
llm_bind = llm.bind_tools([write_email, schedule_meeting, check_calendar_availability, Done], tool_choice="any", parallel_tool_calls=True)

In [110]:
llm_bind.invoke("Schedule a calender meeting and check availablity").tool_calls

[{'name': 'check_calendar_availability',
  'args': {'day': 'today'},
  'id': 'call_NuHEmgJi9UHJGebfNH3oFelS',
  'type': 'tool_call'}]

In [17]:
from langgraph.graph import MessagesState

class State(MessagesState):
    email_input : dict
    classification_decision: Literal["ignore", "notify", "respond"]


In [19]:
from pydantic import Field
class RouterSchema(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [31]:
tools = [write_email, schedule_meeting, check_calendar_availability, Done]
tools_by_name = {tool.name: tool for tool in tools}

tools_by_name

{'write_email': StructuredTool(name='write_email', description='Write and send email', args_schema=<class 'langchain_core.utils.pydantic.write_email'>, func=<function write_email at 0x12ce3c860>),
 'schedule_meeting': StructuredTool(name='schedule_meeting', description='Schedule a calender meeting', args_schema=<class 'langchain_core.utils.pydantic.schedule_meeting'>, func=<function schedule_meeting at 0x12ce3ccc0>),
 'check_calendar_availability': StructuredTool(name='check_calendar_availability', description='Check calendar availability for a given day.', args_schema=<class 'langchain_core.utils.pydantic.check_calendar_availability'>, func=<function check_calendar_availability at 0x12ce3c9a0>),
 'Done': StructuredTool(name='Done', description='Email has been send', args_schema=<class 'langchain_core.utils.pydantic.Done'>, func=<class '__main__.Done'>)}

In [78]:
from operator import itemgetter
from langgraph.graph import END
from langgraph.types import Command

llm_with_tools = llm.bind_tools(tools,  tool_choice="any")
llm_router = llm.with_structured_output(RouterSchema) 


def get_user_prompt(sender:str, receiver:str, subject:str, thread:str)->str:
    return  f"""Please determine how to handle the below email thread:


        Sender : {sender}

        Receiver : {receiver}

        Subject : {subject}

        Thread : {thread}

    """


def traige_router(state:State)-> Command[Literal["response_agent", "__end__"]]:
    email_input = state["email_input"]
    # destructure the email
    sender, receiver, subject, thread = itemgetter("from", "to", "subject", "thread")(email_input)
    # create user prompt
    u_prompt = get_user_prompt(sender=sender, receiver=receiver, subject=subject, thread=thread)
    ## invoke the llm
    response = llm_router.invoke(u_prompt)

    if response.classification == "ignore":
        goto = END
        update = {
            "classification_decison" : "ignore"
        }
    elif response.classification == "notify":
        goto=END
        update = {
            "classification_decision" : "notify"
        }
    elif response.classification == "respond":
        goto= "response_agent"
        update = {
            "classification_decision" : "respond",
            "messages": {
                "role": "user",
                "content": u_prompt
            }
        }
    else:
        return ValueError("Invalid value")
    
    return Command(goto=goto, update=update)
    
    

In [79]:
def call_llm(state:State):
    """LLM decide whther to call the tools or not"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": response}

In [80]:
from langgraph.graph import StateGraph, START, END

agent = StateGraph(State)
agent.add_node("call_llm", call_llm)
agent.add_edge(START, "call_llm")

agent = agent.compile()

In [81]:
email_input = {
  "from": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}

In [82]:
from langgraph.graph import StateGraph, START, END

test = StateGraph(State)
test.add_node("triage", traige_router)
test.add_node("response_agent", agent)
test.add_edge(START, "triage")


test = test.compile()

In [83]:
email_input

{'from': 'Alice Smith <alice.smith@company.com>',
 'to': 'John Doe <john.doe@company.com>',
 'subject': 'Quick question about API documentation',
 'thread': "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"}

In [84]:
outs = test.invoke({"email_input": email_input})
outs

{'messages': [HumanMessage(content="Please determine how to handle the below email thread:\n\n\n        Sender : Alice Smith <alice.smith@company.com>\n\n        Receiver : John Doe <john.doe@company.com>\n\n        Subject : Quick question about API documentation\n\n        Thread : Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice\n\n    ", additional_kwargs={}, response_metadata={}, id='cb06e123-7d4c-443a-a569-366348d4a814'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2805, 'prompt_tokens': 338, 'total_tokens': 3143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2432, 'rejected_prediction_tokens': 0}, 

In [86]:
outs["messages"][-1].tool_calls

[{'name': 'write_email',
  'args': {'to': 'alice.smith@company.com',
   'subject': 'Re: Quick question about API documentation',
   'thread': "Hi Alice,\n\nThanks for flagging this. You're right—/auth/refresh and /auth/validate seem to be missing from the current API docs for the authentication service.\n\nProposed resolution:\n- Confirm with the API platform team whether these endpoints exist and their intended usage.\n- If they exist: add to the docs with endpoint details (HTTP method, path, required headers, sample requests/responses, error handling, auth requirements). Include any versioning notes.\n- If they do not exist: update the docs to reflect that these endpoints are not yet released or remove references; provide ETA if possible.\n- Update the documentation changelog and notify stakeholders (docs, QA, backend, product).\n- Share a revised doc link or PR/status update as soon as ready.\n\nProposed next steps I can take:\n1) Check with the API service owner about the endpoints